In [ ]:
# Import libararies
from financetoolkit import Toolkit

In [ ]:
# Load tickers
tick_path = "./stock_lists/wilshire_5000.txt" 
tickers = []

with open(tick_path, "r") as file:
    
    # Read each line in the file and append it to the list
    for line in file:
        tickers.append(line.strip())

# Print our list of tickers
print(f"Loading data from {len(tickers)} stocks...")

In [ ]:
# Use a private, Basic-Plan api key to fetch all quarterly data between 2001 and 2023.
key = "[API_KEY]"
companies = Toolkit(tickers, api_key=key, start_date='2001-01-01', end_date='2023-01-01', quarterly=True)

In [ ]:
# Get distorical data and manipulate it into the standard format
historical_data = companies.get_historical_data(period="quarterly")
historical_data = historical_data.stack(level=1)

In [ ]:
# Define a custom operation for calculating stock returns over the next quarter
def calculateReturns(group):
    group['Return_Lag'] = (group['Close'].shift(-1) - group['Close']) / group['Close'].shift(-1)
    return group


In [ ]:
# Copy over the historial return data
data = historical_data
data = data.sort_index(level=1)

# Calculate stock grwoth over the next quarter
new_data = data.groupby(level=1).apply(calculateReturns)

# Isolate the single column associated with stock returns
new_data = new_data.sort_index(level=1)
new_data = new_data.droplevel(2)
new_data = new_data["Return_Lag"]

In [ ]:
# Print the head of our final dataset
print(data.head())

In [ ]:
# Output the dataset
data.to_csv('./output/return.csv')